In [ ]:
!sudo apt update
!sudo apt install -y tesseract-ocr
!pip install pytesseract pillow python-docx langchain langchain-community faiss-cpu sentence-transformers

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,665 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,717 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [ ]:
from google.colab import files
uploaded = files.upload()  # drag and drop your HR.docx here
docx_path = next(iter(uploaded))  # gets the filename

Saving Sample_BRD_Policy_Management_System.docx to Sample_BRD_Policy_Management_System.docx


In [ ]:
!pip install docx2txt





In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader(docx_path)
docs_text = loader.load()
doc_text = "\n".join([doc.page_content for doc in docs_text])


In [ ]:
from docx import Document

In [ ]:
def extract_tables(docx_path):
    doc = Document(docx_path)
    table_data = []
    for table in doc.tables:
        for row in table.rows:
            row_data = [cell.text.strip() for cell in row.cells]
            if any(row_data):
                table_data.append(" | ".join(row_data))
    return "\n".join(table_data)

tables_text = extract_tables(docx_path)


In [ ]:
#OCR image content from .docx using Tesseract
def extract_images_and_ocr(docx_path, lang='eng'):
    image_texts= []
    success_count = 0  # Count successful OCRs
    try:
        with ZipFile(docx_path) as z:
            for file in z.namelist():
                if file.startswith("word/media/") and file.lower().endswith((".png", ".jpg", ".jpeg")):
                    print(f"🔍 Processing image: {file}")
                    image_bytes = z.read(file)
                    image = Image.open(BytesIO(image_bytes))

                    text = pytesseract.image_to_string(image, lang=lang)

                    if text.strip():
                        print(f"✅ OCR text extracted from {file}")
                        image_texts.append(f"[{file}]\n{text.strip()}")
                        success_count += 1
                    else:
                        print(f"⚠️ No text found in {file}")

        print(f"\n📊 Total images processed: {success_count} with text extracted")
        return "\n\n".join(image_texts)

    except Exception as e:
        print("❌ Error extracting images or running OCR:", e)
        return ""


In [ ]:
full_text = f"{doc_text}\n\n[TABLE DATA]\n{tables_text}\n\n[IMAGE OCR TEXT]\n{image_texts}"

NameError: name 'image_texts' is not defined

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.create_documents([full_text])

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunks, embedding_model)
print("✅ FAISS vector store created")

In [ ]:
query = input(" Ask something from your HR document: ")
results = db.similarity_search(query, k=1)

In [ ]:
print("\n📄 Top Matching Chunks:\n")
for i, res in enumerate(results, 1):
    print(f"{i}. {res.page_content}\n")